In [1]:
from langchain_groq import ChatGroq




In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content

# Remove extra spaces and line breaks
cleaned_data = re.sub(r'\s+', ' ', page_data).strip()

print(cleaned_data)

Nike Careers Skip to main content Open Virtual Assistant Home Career Areas Total Rewards Life@Nike Purpose Language Select a Language Deutsch English Español (España) Español (América Latina) Français Italiano Nederlands Polski Tiếng Việt Türkçe 简体中文 繁體中文 עִברִית 한국어 日本語 Careers Close Menu Careers Chat Home Career Areas Total Rewards Life@Nike Purpose Jordan Careers Converse Careers Language Menu Return to Previous Menu Select a Language Deutsch English Español (España) Español (América Latina) Français Italiano Nederlands Polski Tiếng Việt Türkçe 简体中文 繁體中文 עִברִית 한국어 日本語 Your browser doesn't support HTML5 video tag. Play Full Video WHAT MOVES YOUMOVES OUR WORLD Pause Video Play Video Play 'How We Do It' video Play 'Victory Mode' video' Play 'Craft the Future' video Be In the Know Retail at NIKE, Inc. Learn More The Nike Internship Experience Learn More AI/ML Roles for Innovators Learn More Peek Into Our World Learn More NikeUNITED's Commitment Learn More IF YOU HAVE A BODY, YOU ARE A

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key="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"
)

page_data = """
We are hiring a Data Scientist.
Experience: 2-4 years.
Skills: Python, Machine Learning, SQL.
Description: Work on predictive modeling, analytics, and AI solutions.
"""

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}

    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: 'role', 'experience', 'skills' and 'description'.

    Only return valid JSON.
    """
)

chain_extract = prompt_extract | llm

res = chain_extract.invoke({"page_data": page_data})

print(res.content)

```json
{
  "role": "Data Scientist",
  "experience": "2-4 years",
  "skills": ["Python", "Machine Learning", "SQL"],
  "description": "Work on predictive modeling, analytics, and AI solutions."
}
```


In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Lead, Digital Site Operations, Nike Direct Digital Commerce',
  'experience': None,
  'skills': None,
  'description': 'Job Location: Karnataka, Karnātaka, India, Business Operations'},
 {'role': 'Senior Product Manager - Transportation Management System (TMS)',
  'experience': None,
  'skills': None,
  'description': 'Job Location: Karnataka, Karnātaka, India, Program, Product & Project Management'},
 {'role': 'Software Engineer II',
  'experience': None,
  'skills': None,
  'description': 'Job Location: Karnataka, Karnātaka, India, Software Engineering'},
 {'role': 'Store Manager - Nike Oxmoor',
  'experience': None,
  'skills': None,
  'description': 'Job Location: Louisville, Kentucky, United States, Retail Stores'},
 {'role': 'Athlete 16PT (교하)',
  'experience': None,
  'skills': None,
  'description': 'Job Location: Gyeonggi-do, Gyeonggi-do, South Korea, Retail Stores'},
 {'role': 'Athlete II',
  'experience': None,
  'skills': None,
  'description': 'Job Location: Gua

In [12]:
type(json_res)

list

In [13]:
import pandas as pd

df = pd.read_excel("my_portfolio.xlsx")

df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular, .NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [14]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"], 
                       metadatas={"links": row["Links"]}, 
                       ids=[str(uuid.uuid4())])

In [15]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React Native"], n_results=2)
links

{'ids': [['07804dbe-34c6-426f-b1c8-e7b258f9b33a',
   'acc2a96a-5f15-4a8a-b597-4752ae555f5f'],
  ['597461b3-466e-49ac-be1f-fccd5752dc10',
   'b1142e4e-2c69-4aa1-bf47-4adca2234910']],
 'embeddings': None,
 'documents': [['Machine Learning, Python, TensorFlow',
   'Python, Django, MySQL'],
  ['React Native, Node.js, MongoDB', 'React, Node.js, MongoDB']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/python-portfolio'}],
  [{'links': 'https://example.com/react-native-portfolio'},
   {'links': 'https://example.com/react-portfolio'}]],
 'distances': [[0.996884822845459, 1.0578701496124268],
  [0.9917072653770447, 1.1951290369033813]]}

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

# ✅ PUT YOUR REAL API KEY HERE
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key="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"   # ← Replace with real key
)

# ✅ Define job
job = """
Role: Python Developer
Skills Required: Python, Django, REST API, SQL
Experience: 2+ years backend development
"""

# ✅ Define links
links = """
1. https://atliq-ai-portfolio.com
2. https://atliq-automation-case-study.com
3. https://atliq-ml-projects.com
"""

# ✅ Prompt
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting 
        the seamless integration of business processes through automated tools.
        Over our experience we have empowered numerous enterprises with tailored solutions, fostering
        processes optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above and how we can assist them 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase AtliQ's portfolio.
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)

chain_email = prompt_email | llm

res = chain_email.invoke({
    "job_description": job,
    "link_list": links
})

print(res.content)

Subject: Expert Python Development Services for Your Business Needs

Dear Hiring Manager,

I came across your job posting for a Python Developer and was impressed by the role's requirements. As a Business Development Executive at AtliQ, I'd like to introduce our company as a potential partner in fulfilling your Python development needs.

At AtliQ, we specialize in providing AI and software consulting services, focusing on the seamless integration of business processes through automated tools. With our expertise in Python, Django, REST API, and SQL, we can assist you in finding the perfect candidate or even take on the development project ourselves.

Our team has a proven track record of delivering high-quality solutions, optimizing processes, reducing costs, and enhancing overall efficiency for numerous enterprises. Some of our notable projects include:

* Developing a predictive maintenance platform using machine learning and Python for a leading manufacturing company (https://atliq.c